# Creating the Tools with Standalone Callbacks 
Goal: create the tools so that people can use them as a python script

Steps to complete the main tool:
1. set up changeable channels with the most basic barebone example
2. link the two plots together
3. link to the frequency slider (mind the javacall back: callback_policy="mouseup")
4. set up histogram and bin changer
5. link histogram data. make sure it can be updated
6. fix minor details such as plot labels etc.

#### Barebone example: PSD plot with changeable channels

In [2]:
import sys
# locate your spectralCV so we have scv_funcs to use
sys.path.append('/Users/ldliao/Research/Projects/spectralCV/')
sys.path.append('/Users/ldliao/Research/Projects/spectralCV/notebooks_visualization/')

In [3]:
import os
# the only lines you need to change to inspect different saved data
path = '/Users/ldliao/Research/Projects/spectralCV/results/kjm_digits/bp/'
os.chdir(path)

In [4]:
# imports
import numpy as np
import scipy as sp
from scipy.stats import expon
import glob

import neurodsp as ndsp
from scv_funcs import lfpca
import warnings
warnings.filterwarnings('ignore')

In [5]:
# bokeh imports
import bokeh
from bokeh.io import push_notebook, show, output_notebook, curdoc
from bokeh.io import *
from bokeh.plotting import figure, output_file, show
from bokeh.layouts import gridplot, row, column
from bokeh.models import ColumnDataSource, TapTool, Slider, Span, CustomJS
from bokeh.models.widgets import PreText, Select, Slider
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

output_notebook()

Loading BokehJS ...

In [6]:
import os
# the only lines you need to change to inspect different saved data
path = '/Users/ldliao/Research/Projects/spectralCV/results/kjm_digits/bp/'
os.chdir(path)

In [7]:
# collect all the names of .npz files in the folder
lfpca_all_names = glob.glob("*.npz")
lfpca_all_names.sort()

# loading the npz files 
lfpca_all = {}
for ind, lf in enumerate(lfpca_all_names):
    lfpca_all[lf[:-4]] = lfpca.lfpca_load_spec(lf)

# initialize with the first lfpca object
lf = lfpca_all[lfpca_all_names[0][:-4]]

# grabbing channel count from psd
chan_count, freq = lf.psd.shape

# mapping all the channels
DEFAULT_TICKERS = list(map(str, range(chan_count)))
LF_TICKERS = [key for key in lfpca_all.keys()]

In [105]:
# setting up the defaults
chan = 0
select_freq = 10
select_bin = 20
freq_vals = lf.f_axis[1:]
psd_vals = lf.psd[chan].T[1:]
scv_vals = lf.scv[chan].T[1:]

attr = ColumnDataSource(data=dict(chan=[chan]))
source = ColumnDataSource(data=dict(freq_vals=freq_vals, 
                                    psd_vals=psd_vals, 
                                    scv_vals=scv_vals))

In [106]:
ch_ticker = Select(value=str(chan), title='channel', options=DEFAULT_TICKERS)

In [107]:
psd_plot = figure(title='PSD', x_axis_type='log', y_axis_type='log')
psd_plot.legend.location = 'top_left'
psd_plot.xaxis.axis_label = 'Frequency (Hz)'
psd_plot.yaxis.axis_label = 'Power/Frequency (dB/Hz)'
psd_plot.grid.grid_line_alpha=0.3

In [108]:
# customize plot to psd
def create_psd_plot(psd_plot, source):
    psd_plot.line('freq_vals', 'psd_vals', source=source, color='navy')

In [109]:
lf.f_axis[1:]

array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
        12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
        23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,
        34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,
        45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,
        56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,
        67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,
        78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,
        89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99.,
       100., 101., 102., 103., 104., 105., 106., 107., 108., 109., 110.,
       111., 112., 113., 114., 115., 116., 117., 118., 119., 120., 121.,
       122., 123., 124., 125., 126., 127., 128., 129., 130., 131., 132.,
       133., 134., 135., 136., 137., 138., 139., 140., 141., 142., 143.,
       144., 145., 146., 147., 148., 149., 150., 15

In [110]:
create_psd_plot(psd_plot, source)

In [111]:
widgets = row(ch_ticker)
layout = column(widgets, psd_plot)

chan_callback = CustomJS(args=dict(attr=attr, source=source), code="""
    var attr_data = attr.data
    attr_data['chan'] = cb_obj.value
    var f = cb_obj.value
    
    var data = source.data
    data['freq_vals'] = lf.f_axis[1,]
    data['psd_vals'] = lf.psd.f[[1,]]
    data['psd_vals'] = lf.scv.f[[1,]]

    source.change.emit();
    attr.change.emit();
""")

ch_ticker.js_on_change('value', chan_callback)

# In the notebook, just pass the function that defines the app to show
show(layout)

In [10]:
# imports
from bokeh.io import show, output_file, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.layouts import column, widgetbox
from bokeh.models import CustomJS, Select

# defaults
select_freq = 20

# set up data source as a dictionary
data_source = {
    'freq_vals': lf.f_axis[1:],
    'psd_vals': lf.psd[int(DEFAULT_TICKERS[0])].T[1:],
    'scv_vals': lf.scv[int(DEFAULT_TICKERS[0])].T[1:]
}    

# populate the data dictionary
for tick in DEFAULT_TICKERS:
    psd_ch_str = 'ch' + tick + "_psd_vals"
    scv_ch_str = 'ch' + tick + "_scv_vals"
    data_source[psd_ch_str] = lf.psd[int(tick)].T[1:]
    data_source[scv_ch_str] = lf.scv[int(tick)].T[1:]

# link dictionary as data source
source = ColumnDataSource(data=data_source)

# set up ticker
ch_ticker = Select(value=DEFAULT_TICKERS[0], title='channel', options=DEFAULT_TICKERS)

# write javascript callback
js_str = "data = source.data; f = cb_obj.value; "
chan = DEFAULT_TICKERS[0]
for tick in DEFAULT_TICKERS:
    psd_ch_str = 'ch' + tick + "_psd_vals"
    scv_ch_str = 'ch' + tick + "_scv_vals"
    if tick == DEFAULT_TICKERS[0]:
        js_str = js_str + "if (f == " + tick + ") {data['psd_vals'] = data." + psd_ch_str +";" 
        js_str = js_str + " data['scv_vals'] = data." + scv_ch_str +";}"
    else:
        js_str = js_str + " else if (f == " + tick + ") { data['psd_vals'] = data." + psd_ch_str +";"
        js_str = js_str + " data['scv_vals'] = data." + scv_ch_str +";}"
        
js_str = js_str + " source.change.emit();"

# javascript on change for ticker
ch_callback = CustomJS(args=dict(source=source), code=js_str)
ch_ticker.js_on_change('value', ch_callback) 

# set up slider
freq_slider = Slider(start=1, end=199, value=select_freq, step=1, title="Frequency", callback_policy="mouseup")

# set up psd plot
psd_plot = figure(title='PSD', x_axis_type='log', y_axis_type='log', plot_width=300, plot_height=300)
psd_plot.legend.location = 'top_left'
psd_plot.xaxis.axis_label = 'Frequency (Hz)'
psd_plot.yaxis.axis_label = 'Power/Frequency (dB/Hz)'
psd_plot.grid.grid_line_alpha=0.3
psd_plot.line('freq_vals', 'psd_vals', source=source)    

# set up scv plot
scv_plot = figure(title='SCV', x_axis_type='log', y_axis_type='log', plot_width=300, plot_height=300)
scv_plot.legend.location='top_left'
scv_plot.xaxis.axis_label = 'Frequency (Hz)'
scv_plot.yaxis.axis_label = '(Unitless)'
scv_plot.grid.grid_line_alpha=0.3
fit_line = bokeh.models.glyphs.Line(x='freq_vals', y=1, line_width=5, line_alpha=0.5, line_color='darkgrey')
scv_plot.add_glyph(source, fit_line)
scv_plot.line('freq_vals', 'scv_vals', source=source, color='navy')

# add in frequency slider vertical lines
vline_psd = Span(location=select_freq, dimension='height', line_color='red', line_dash='dashed', line_width=3)
vline_scv = Span(location=select_freq, dimension='height', line_color='red', line_dash='dashed', line_width=3)
psd_plot.add_layout(vline_psd)
scv_plot.add_layout(vline_scv)

# set up connector spans
freq_slider.callback = CustomJS(args=dict(span1 = vline_psd,
                                          span2 = vline_scv,
                                          slider = freq_slider),
                                          code = """span1.location = slider.value; 
                                                    span2.location = slider.value""")
    
# set up desired layout
widgets = row(ch_ticker, freq_slider)
# sliders = row(freq_slider, bin_slider)
layout = column(widgets, row(psd_plot, scv_plot))
show(layout)

In [99]:
# imports
from bokeh.io import show, output_file, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.layouts import column, widgetbox
from bokeh.models import CustomJS, Select

# defaults
select_freq = 20

# set up data source as a dictionary
data_source = {
    'freq_vals': lf.f_axis[1:],
    'psd_vals': lf.psd[int(DEFAULT_TICKERS[0])].T[1:],
    'scv_vals': lf.scv[int(DEFAULT_TICKERS[0])].T[1:]
}    

# populate the data dictionary
for tick in DEFAULT_TICKERS:
    psd_ch_str = 'ch' + tick + "_psd_vals"
    scv_ch_str = 'ch' + tick + "_scv_vals"
    data_source[psd_ch_str] = lf.psd[int(tick)].T[1:]
    data_source[scv_ch_str] = lf.scv[int(tick)].T[1:]

# link dictionary as data source
source = ColumnDataSource(data=data_source)

# set up ticker
ch_ticker = Select(value=DEFAULT_TICKERS[0], title='channel', options=DEFAULT_TICKERS)

# write javascript callback
js_str = "data = source.data; f = cb_obj.value; "
chan = DEFAULT_TICKERS[0]
for tick in DEFAULT_TICKERS:
    psd_ch_str = 'ch' + tick + "_psd_vals"
    scv_ch_str = 'ch' + tick + "_scv_vals"
    if tick == DEFAULT_TICKERS[0]:
        js_str = js_str + "if (f == " + tick + ") {data['psd_vals'] = data." + psd_ch_str +";" 
        js_str = js_str + " data['scv_vals'] = data." + scv_ch_str +";}"
    else:
        js_str = js_str + " else if (f == " + tick + ") { data['psd_vals'] = data." + psd_ch_str +";"
        js_str = js_str + " data['scv_vals'] = data." + scv_ch_str +";}"
        
js_str = js_str + " source.change.emit();"

# javascript on change for ticker
ch_callback = CustomJS(args=dict(source=source), code=js_str)
ch_ticker.js_on_change('value', ch_callback) 

# set up slider
freq_slider = Slider(start=1, end=199, value=select_freq, step=1, title="Frequency", callback_policy="mouseup")

# set up psd plot
psd_plot = figure(title='PSD', x_axis_type='log', y_axis_type='log', plot_width=300, plot_height=300)
psd_plot.legend.location = 'top_left'
psd_plot.xaxis.axis_label = 'Frequency (Hz)'
psd_plot.yaxis.axis_label = 'Power/Frequency (dB/Hz)'
psd_plot.grid.grid_line_alpha=0.3
psd_plot.line('freq_vals', 'psd_vals', source=source)    

# set up scv plot
scv_plot = figure(title='SCV', x_axis_type='log', y_axis_type='log', plot_width=300, plot_height=300)
scv_plot.legend.location='top_left'
scv_plot.xaxis.axis_label = 'Frequency (Hz)'
scv_plot.yaxis.axis_label = '(Unitless)'
scv_plot.grid.grid_line_alpha=0.3
fit_line = bokeh.models.glyphs.Line(x='freq_vals', y=1, line_width=5, line_alpha=0.5, line_color='darkgrey')
scv_plot.add_glyph(source, fit_line)
scv_plot.line('freq_vals', 'scv_vals', source=source, color='navy')

# add in frequency slider vertical lines
vline_psd = Span(location=select_freq, dimension='height', line_color='red', line_dash='dashed', line_width=3)
vline_scv = Span(location=select_freq, dimension='height', line_color='red', line_dash='dashed', line_width=3)
psd_plot.add_layout(vline_psd)
scv_plot.add_layout(vline_scv)

# set up connector spans
freq_slider.callback = CustomJS(args=dict(span1 = vline_psd,
                                          span2 = vline_scv,
                                          slider = freq_slider),
                                          code = """span1.location = slider.value; 
                                                    span2.location = slider.value""")
    
# set up desired layout
widgets = row(ch_ticker, freq_slider)
# sliders = row(freq_slider, bin_slider)
layout = column(widgets, row(psd_plot, scv_plot, hist_fig))
show(layout)

In [100]:
chan = int(DEFAULT_TICKERS[0])
select_bin = 20
select_freq = 10
bin_slider = Slider(start=10, end=55, value=select_bin, step=5, title="Number of bins", callback_policy="mouseup")
attr = ColumnDataSource({'chan': [chan], 'select_bin':[select_bin], 'select_freq':[select_freq]})

# create histogram frame
hist_source = ColumnDataSource({'top': [], 'left': [], 'right': []})
fit_hist_source = ColumnDataSource({'x': [], 'y': []})
hist, edges = np.histogram(lf.spg[int(ch_ticker.value), freq_slider.value, :], bins=bin_slider.value, density=True)
hist_source.data = {'top': hist, 'left': edges[:-1], 'right': edges[1:]}

# create fit line for the histogram
rv = expon(scale=sp.stats.expon.fit(lf.spg[chan,select_freq,:],floc=0)[1])
hist_source.data = {'top': hist, 'left': edges[:-1], 'right': edges[1:]}
fit_hist_source.data = {'x': edges, 'y': rv.pdf(edges)}

hist_fig = figure(x_axis_label='Power', 
                  y_axis_label='Probability',plot_width=300, plot_height=300)
hist_fig.axis.visible = False
hist_fig.title.text = 'Freq = %.1fHz, p-value = %.4f'%(select_freq, lf.ks_pvals[chan, select_freq])
hist_fig.quad(top='top', bottom=0, left='left', right='right', source=hist_source, fill_color='#295B99', line_color="#033649")


bokeh.models.renderers.GlyphRenderer(
    id='477f1461-c45b-4a97-a417-48e3e07d56e8',
    data_source=bokeh.models.sources.ColumnDataSource(
        id='d5d23326-2251-4374-b2b4-8d9541d1267b',
        callback=None,
        data={'top': {'__ndarray__': 'vShbEfX1FT+epkoFFH8GP+Cqax3WbAU/V6Ip7VGR9z7Wmee8zbXZPtyZ57zNtdk+0ZnnvM212T4AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAADou+993iPBPg==',
          'shape': (20,),
          'dtype': 'float64'},
         'left': {'__ndarray__': 'fg8kUbaMTkDiMjSDsMyyQNIO48wjrsJAMwQsWO/1y0DLfLpx3Z7SQHz3XjfDQtdALHID/ajm20Bv9lNhR0XgQMczJkQ6l+JAH3H4Ji3p5EB4rsoJIDvnQNDrnOwSjelAKClvzwXf60CBZkGy+DDuQO3Ricp1QfBAmfDyO29q8UBFD1ytaJPyQPEtxR5ivPNAnUwukFvl9EBKa5cBVQ72QA==',
          'shape': (20,),
          'dtype': 'float64'},
         'right': {'__ndarray__': '4jI0g7DMskDSDuPMI67CQDMELFjv9ctAy3y6cd2e0kB89143w0LXQCxyA/2o5ttAb/ZTYUdF4EDHMyZEOpfiQB9x+CYt6eRAeK7KCSA750DQ65zsEo3pQCgpb88F3+tAgWZBsvgw7kDt0YnKdUHwQJnw8jtvavFARQ9crWiT8kDxLcUeYrzzQJ1MLpBb5fRASmuXAVUO9kD2iQBzTjf3QA==',
          'shape': (20,),
          'dtype': 'float64'}},
        js_event_callbacks={},
        js_property_callbacks={},
        name=None,
        selected=bokeh.models.selections.Selection(
            id='ad72f01b-33ac-4a12-bd37-e728000ed2e5',
            indices=[],
            js_event_callbacks={},
            js_property_callbacks={},
            line_indices=[],
            multiline_indices={},
            name=None,
            subscribed_events=[],
            tags=[]),
        selection_policy=bokeh.models.selections.UnionRenderers(
            id='1e9b1b12-c280-4e64-8cf8-b00f00b1d8fe',
            js_event_callbacks={},
            js_property_callbacks={},
            name=None,
            subscribed_events=[],
            tags=[]),
        subscribed_events=[],
        tags=[]),
    glyph=bokeh.models.glyphs.Quad(
        id='52262ab5-6215-419d-87ac-f17fd29253f6',
        bottom={'value': 0},
        fill_alpha={'value': 1.0},
        fill_color={'value': '#295B99'},
        js_event_callbacks={},
        js_property_callbacks={},
        left={'field': 'left'},
        line_alpha={'value': 1.0},
        line_cap='butt',
        line_color={'value': '#033649'},
        line_dash=[],
        line_dash_offset=0,
        line_join='bevel',
        line_width={'value': 1},
        name=None,
        right={'field': 'right'},
        subscribed_events=[],
        tags=[],
        top={'field': 'top'}),
    hover_glyph=None,
    js_event_callbacks={},
    js_property_callbacks={},
    level='glyph',
    muted=False,
    muted_glyph=None,
    name=None,
    nonselection_glyph=bokeh.models.glyphs.Quad(
        id='d6b1897f-d743-4db5-958d-187e0fdf4ec4',
        bottom={'value': 0},
        fill_alpha={'value': 0.1},
        fill_color={'value': '#1f77b4'},
        js_event_callbacks={},
        js_property_callbacks={},
        left={'field': 'left'},
        line_alpha={'value': 0.1},
        line_cap='butt',
        line_color={'value': '#1f77b4'},
        line_dash=[],
        line_dash_offset=0,
        line_join='bevel',
        line_width={'value': 1},
        name=None,
        right={'field': 'right'},
        subscribed_events=[],
        tags=[],
        top={'field': 'top'}),
    selection_glyph=None,
    subscribed_events=[],
    tags=[],
    view=bokeh.models.sources.CDSView(
        id='695d5c96-4597-418f-a357-3615332a2322',
        filters=[],
        js_event_callbacks={},
        js_property_callbacks={},
        name=None,
        source=bokeh.models.sources.ColumnDataSource(
            id='d5d23326-2251-4374-b2b4-8d9541d1267b',
            callback=None,
            data={'top': {'__ndarray__': 'vShbEfX1FT+epkoFFH8GP+Cqax3WbAU/V6Ip7VGR9z7Wmee8zbXZPtyZ57zNtdk+0ZnnvM212T4AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAADou+993iPBPg==',
              'sh

In [101]:
hist, edges = np.histogram(lf.spg[int(chan), select_freq, :], bins=select_bin, density=True)
rv = expon(scale=sp.stats.expon.fit(lf.spg[int(chan),select_freq,:],floc=0)[1])
hist_source.data = {'top': hist, 'left': edges[:-1], 'right': edges[1:]}
fit_hist_source.data = {'x': edges, 'y': rv.pdf(edges)}

hist_fig.title.text = 'Freq = %.1fHz, p-value = %.4f'%(freq_slider.value, lf.ks_pvals[int(ch_ticker.value), freq_slider.value])
fit_line = bokeh.models.glyphs.Line(x='x', y='y', line_width=8, line_alpha=0.7, line_color="#D53B54")
hist_fig.add_glyph(fit_hist_source, fit_line)

bokeh.models.renderers.GlyphRenderer(
    id='57384b6f-862a-4833-8929-3847fae41ba8',
    data_source=bokeh.models.sources.ColumnDataSource(
        id='12f74d7f-9b5d-4cf8-8b4d-e802f89b76b0',
        callback=None,
        data={'y': {'__ndarray__': 'y5Wf4F8G7T6FxDfIgmfgPiDmrPH4itI+gcishs31xD6r8XJRTrG3Pg9apJv/x6o+sKiKqMRFnj5O8xS7BhyRPhdUpToFV4M+EtnDf3PcdT6apzC2BbZoPv7f1uGz7ls+jCdzFuSSTz4h0BIYTdhBPhOSTeLWKzQ+OqE1jgPNJj5LnKwP8sUZPo4PXx4TIg0++N/9nip3AD7FKPNBq5zyPTki+2zOCeU9',
          'shape': (21,),
          'dtype': 'float64'},
         'x': {'__ndarray__': 'UnAD+oTGckC0F3s5ozHkQEMUgbTcHvRArJxEzOck/kCLEgRyeRUEQcDW5f1+GAlB9JrHiYQbDkGVr9QKRY8RQa+RxdDHEBRByXO2lkqSFkHkVadczRMZQf43mCJQlRtBGBqJ6NIWHkEZ/jzXKkwgQSdvNTrsjCFBNOAtna3NIkFBUSYAbw4kQU7CHmMwTyVBWzMXxvGPJkFppA8ps9AnQXYVCIx0ESlB',
          'shape': (21,),
          'dtype': 'float64'}},
        js_event_callbacks={},
        js_property_callbacks={},
        name=None,
        selected=bokeh.models.selections.Selection(
            id='d0b4a5a3-86da-4dcb-acf8-950bad25080c',
            indices=[],
            js_event_callbacks={},
            js_property_callbacks={},
            line_indices=[],
            multiline_indices={},
            name=None,
            subscribed_events=[],
            tags=[]),
        selection_policy=bokeh.models.selections.UnionRenderers(
            id='8015b339-8ee3-4e46-9ccc-f6b50ac1459a',
            js_event_callbacks={},
            js_property_callbacks={},
            name=None,
            subscribed_events=[],
            tags=[]),
        subscribed_events=[],
        tags=[]),
    glyph=bokeh.models.glyphs.Line(
        id='1356c9ae-d797-46b2-8dd3-4a4346e7b33a',
        js_event_callbacks={},
        js_property_callbacks={},
        line_alpha=0.7,
        line_cap='butt',
        line_color='#D53B54',
        line_dash=[],
        line_dash_offset=0,
        line_join='bevel',
        line_width=8,
        name=None,
        subscribed_events=[],
        tags=[],
        x={'field': 'x'},
        y={'field': 'y'}),
    hover_glyph=None,
    js_event_callbacks={},
    js_property_callbacks={},
    level='glyph',
    muted=False,
    muted_glyph=None,
    name=None,
    nonselection_glyph='auto',
    selection_glyph='auto',
    subscribed_events=[],
    tags=[],
    view=bokeh.models.sources.CDSView(
        id='12515fbd-c956-490a-b2d4-042924848d92',
        filters=[],
        js_event_callbacks={},
        js_property_callbacks={},
        name=None,
        source=bokeh.models.sources.ColumnDataSource(
            id='12f74d7f-9b5d-4cf8-8b4d-e802f89b76b0',
            callback=None,
            data={'y': {'__ndarray__': 'y5Wf4F8G7T6FxDfIgmfgPiDmrPH4itI+gcishs31xD6r8XJRTrG3Pg9apJv/x6o+sKiKqMRFnj5O8xS7BhyRPhdUpToFV4M+EtnDf3PcdT6apzC2BbZoPv7f1uGz7ls+jCdzFuSSTz4h0BIYTdhBPhOSTeLWKzQ+OqE1jgPNJj5LnKwP8sUZPo4PXx4TIg0++N/9nip3AD7FKPNBq5zyPTki+2zOCeU9',
              'shape': (21,),
              'dtype': 'float64'},
             'x': {'__ndarray__': 'UnAD+oTGckC0F3s5ozHkQEMUgbTcHvRArJxEzOck/kCLEgRyeRUEQcDW5f1+GAlB9JrHiYQbDkGVr9QKRY8RQa+RxdDHEBRByXO2lkqSFkHkVadczRMZQf43mCJQlRtBGBqJ6NIWHkEZ/jzXKkwgQSdvNTrsjCFBNOAtna3NIkFBUSYAbw4kQU7CHmMwTyVBWzMXxvGPJkFppA8ps9AnQXYVCIx0ESlB',
              'shape': (21,),
              'dtype': 'float64'}},
            js_event_callbacks={},
            js_property_callbacks={},
            name=None,
            selected=bokeh.models.selections.Selection(
                id='d0b4a5a3-86da-4dcb-acf8-950bad25080c',
                indices=[],
                js_event_callbacks={},
                js_property_callbacks={},
                line_indices=[],
                multiline_indices={},
                name=None,
                subscribed_events=[],
                tags=[]),
            selection_policy=bokeh.models.selections.UnionRenderers(
                id='8015b339-8ee3-4e46-9ccc-f6b50ac1459a',
                js_event_callbacks={},
                js_property_callbacks={},
                name

In [ ]:
# write javascript callback
js_str = "data = source.data; f = cb_obj.value; "
chan = DEFAULT_TICKERS[0]
for tick in DEFAULT_TICKERS:
    psd_ch_str = 'ch' + tick + "_psd_vals"
    scv_ch_str = 'ch' + tick + "_scv_vals"
    if tick == DEFAULT_TICKERS[0]:
        js_str = js_str + "if (f == " + tick + ") {data['psd_vals'] = data." + psd_ch_str +";" 
        js_str = js_str + " data['scv_vals'] = data." + scv_ch_str +";}"
    else:
        js_str = js_str + " else if (f == " + tick + ") { data['psd_vals'] = data." + psd_ch_str +";"
        js_str = js_str + " data['scv_vals'] = data." + scv_ch_str +";}"
        
js_str = js_str + " source.change.emit();"

In [103]:
# set up desired layout
widgets = row(ch_ticker, freq_slider)
# sliders = row(freq_slider, bin_slider)
# layout = column(widgets, row(psd_plot, scv_plot, hist_fig))
layout = column(widgets, row(hist_fig))

show(layout)
# show(hist_fig)